# Implementing a Quantum Algorithm for Monte Carlo Simulations

In this notebook, we attempt to implement parts of the quantum algorithm for Monte Carlo simulations described in "Quantum Computational finance: Monte Carlo pricing of financial derivatives" by Rebentrost, Gupt, and Bromley (2018). We will make reference to the original paper frequently, and the user is encouraged to refer back to it for background and a more theoretical treatment. We were not able to implement the algorithm in full, but we hope the efforts here provide a foundation for future work. 

In [1]:
from pyquil.quil import Program
from pyquil.api import QVMConnection, WavefunctionSimulator
from pyquil.gates import *
from pyquil.quil import address_qubits, DefGate
from pyquil.quilatom import QubitPlaceholder
from pyquil.unitary_tools import program_unitary

from grove.alpha.arbitrary_state import arbitrary_state, unitary_operator
from grove.alpha.phaseestimation.phase_estimation import phase_estimation

import math
import numpy as np
from scipy.stats import norm
from typing import List

In [2]:
qvm = QVMConnection()

## Arithmetic Operations

An important component of the algorithm described in the paper is the use of arithmetic circuits to calculate the payoff of a financial derivative (see Appendix C). Specifically, for a European call option, circuits performing the following are required: addition, addition modulo N, multiplication modulo N, exponentiation modulo N, and max, all of which can be implemented as reversible quantum circuits. Implementing all of these circuits in PyQuil is beyond the scope of this project. Instead, we only implement the addition circuit as a proof of concept and leave implementation of the rest for a future project. 

To implement the adder, we followed the implementation of the plain adder described on page 3 "Quantum Networks for Elementary Arithmetic Operations" by Vedral, Barenco, and Ekert. All references to figures in this section refer to that paper. 

### Carry and Sum

Implementing the plain adder first required implementing the SUM and CARRY gates pictured in Figure 3. A reversed CARRY gate is also implemented. All three functions accept a list of qubit indices and return a program that the performs the operation. 

In [3]:
def quantum_carry_index(q: List[int]) -> Program:
    p = Program()
    p += CCNOT(q[1], q[2], q[3])
    p += CNOT(q[1], q[2])
    p += CCNOT(q[0], q[2], q[3])
    return p

def reverse_carry_index(q: List[int]) -> Program:
    p = Program()
    p += CCNOT(q[0], q[2], q[3])
    p += CNOT(q[1], q[2])
    p += CCNOT(q[1], q[2], q[3])
    return p

def quantum_sum_index(q: List[int]) -> Program:
    p = Program()
    p += CNOT(q[1], q[2])
    p += CNOT(q[0], q[2])
    return p

### Plain Adder

Using the SUM and CARRY gates, we implement the plain adder ADD circuit, which performs the map $|a,b\rangle \rightarrow |a,a+b\rangle$. The adder accepts a program that has prepared a state in the proper encoding for the addition operation, which requires $3n + 1$ qubits, where $n$ is the number of bits needed to represent the numbers. This requires $0 \leq a, b \leq 2^n$. The first $n$ qubits represent $a$, the next $n+1$ qubits represent $b$ with an additional qubit for a carry bit, and the final $n$ qubits are used for the carry register. The implementation follows the circuit diagram in Figure 2.

In [4]:
def quantum_adder_from_state(p: Program, n: int):
    
    a_qubits = [i for i in range(n)]
    b_qubits = [i + n for i in range(n+1)]
    c_qubits = [i + 2*n + 2 for i in range(n)]
        
    for i in range(n):
        if i == n-1: 
            p += quantum_carry_index([c_qubits[i], a_qubits[i], b_qubits[i], b_qubits[i+1]])
        else:
            p += quantum_carry_index([c_qubits[i], a_qubits[i], b_qubits[i], c_qubits[i+1]])
    
    p += CNOT(a_qubits[n-1], b_qubits[n-1])
    p += quantum_sum_index([c_qubits[n-1], a_qubits[n-1], b_qubits[n-1]])
    
    for i in reversed(range(n-1)):
        p += reverse_carry_index([c_qubits[i], a_qubits[i], b_qubits[i], c_qubits[i+1]])
        p += quantum_sum_index([c_qubits[i], a_qubits[i], b_qubits[i]])
    
    return p

To prepare the states, we implement several helper functions. First, int_to_state maps an integer to a quantum state.

In [5]:
# maps int to binary in quantum state. Takes as input a list of qubit placeholders.
# First qubit in the list will be the 0th qubit of the binary representation, so qubits should be sequential.

def int_to_state(a: int, q: List[QubitPlaceholder]) -> Program:
    p = Program()
    binary = format(a, "b")
    emptybits = len(q) - len(binary)
    binary = '0'*emptybits + binary
    for j, bit in enumerate(reversed(binary)):
        if bit =='1':
            p += X(q[j])
        else: 
            p += I(q[j])
    return p

We demonstrate by mapping the integer 3 to a 2 qubit state.

In [6]:
n = 2
a = 3
qubits = QubitPlaceholder.register(n)
pq = int_to_state(a, qubits)
pq = address_qubits(pq)
print(qvm.wavefunction(pq))

(1+0j)|11>


Next, we implement append_integer_to_state, which returns a program to append an additional $n$-bit integer to an $n$-qubit quantum state.

In [7]:
#b is integer to be appended, n is number of bits required for input number
#returns a program that appends binary representation of integer to an n-qubit quantum state

def append_number_to_state(b: int, n: int) -> Program:
    assert b < 2**n, "a uses too many bits"
    
    p = Program()
    state_b = QubitPlaceholder.register(n)
    p += int_to_state(b, state_b)
    p = address_qubits(p, {q: i + n for i, q in enumerate(state_b)})
    
    #print(qvm.wavefunction(p))    
    return p

We demonstrate by appending 1 to the state created above.

In [8]:
b = 1
pq += append_number_to_state(b, n)
print(qvm.wavefunction(pq))

(1+0j)|0111>


Finally, pad_with_zeros adds the carry qubit for integer $b$, the $2$-qubit carry register, and an ancilla qubit to be used later, all initialized to $|0\rangle$.

In [9]:
#pad n-qubit addition state with n + 1 additional qubits initialized to 0
def pad_with_zeros(n: int):
    carry = QubitPlaceholder.register(n+2) #add additional qubit for ancilla
    p = int_to_state(0, carry)
    p = address_qubits(p, {q: i + 2*n for i, q in enumerate(carry)})
    return p

In [10]:
pq += pad_with_zeros(n)
print(qvm.wavefunction(pq))

(1+0j)|00000111>


The state is now ready for the adder. Calling the adder on the program, we see that $1+3=4$ is now represented in binary in the target register.

In [11]:
pq = quantum_adder_from_state(pq, n)
print(qvm.wavefunction(pq))

(1+0j)|00010011>


## Quantum Algorithm for European Option Pricing

From here on out, we implement parts of the quantum algorithm for European option pricing described in Rebentrost. All reference to equations numbers refer back to that paper. In particular, we refer frequently back to sections IV and V. 

### Preparation of quantum state encoding sampling distribution

The first step is to create the probability distribution from which we are sampling. In this case, it's the normal distribution. This will be used to represent the random variable for Brownian motion. To do this, we can use the create_arbitrary_state function from Grove and the procedure described in section V. 

For this example, for simplicity we'll only consider two qubits.

In [12]:
n = 2 #number of qubits needed to represent values

#qubit indices for registers
j_reg = [i for i in range(n)]
control_reg = [i+n for i in range(n+1)]
ancilla = 2*n + 1
carry_reg = [i + 2*n + 2 for i in range(n)]

In [13]:
#optional qubits parameters specifies qubits on which to create probability distribution.

def create_prob_dist(n: int, t: int, xmax: int, qubits: List[int] = None) -> Program:
    #number of qubits to represent 2^n numbers
    t = 4 #period of holding option
    xmax = 4
    dx = 2*xmax/(2**n - 1)
    xj = [(-xmax + j*dx) for j in range(2**n)] #discretization points
    pt = [norm.pdf(x, scale = np.sqrt(t)) for x in xj]
    C = sum(pt)
    pt = pt/C #normalize probabilities 
    p = arbitrary_state.create_arbitrary_state(pt, qubits)

    return p

Let's look at a 2-qubit probability distribution:

In [14]:
t = 4
xmax = 4
p = create_prob_dist(n, t, xmax)
print(qvm.wavefunction(p))

(0.1178392437+0j)|00> + (0.6972186979+0j)|01> + (0.6972186979+0j)|10> + (0.1178392437+0j)|11>


The state is now: $$\mathcal{G}|0^n\rangle = \sum_{j=0}^{2^{n-1}}\sqrt{p_j}|j\rangle$$

We also need the unitary matrix that creates this state for later calculations. We can retrieve the unitary 

Since the adder function that we'll use later requires on $l + 1$ carry qubits, $l$ qubits for the input number, and 1 ancilla qubit, we'll need $3l + 2$ qubits in total. Therefore, the unitaries will need to be of size $2^{3l+2}$, which we accomplish by tensoring $\mathcal{G}$ with $\mathcal{I}_{2^{2l+2}}.$

In [15]:
G = program_unitary(p, n)
Id = np.identity(2**(2*n + 2))
G_unitary = np.kron(G, Id)

We can verify that this unitary creates the desired state by defining it as a new gate in PyQuil.

In [16]:
G_def = DefGate("G", G_unitary)
G = G_def.get_constructor()
pG = Program()
pG += G_def
pG += G(0, 1, 2, 3, 4, 5, 6, 7)
print(qvm.wavefunction(pG))

(0.1178392437+0j)|00000000> + (0.6972186979+0j)|00000001> + (0.6972186979+0j)|00000010> + (0.1178392437+0j)|00000011>


The state is now $$\mathcal{G}|0^n\rangle = \sum_{j=0}^{2^{n-1}}\sqrt{p_j}|j\rangle|0^{l+1}\rangle|0^{l}\rangle$$

# Controlled rotation

The next step is to perform the controlled rotation 

$$\sum_{j=0}^{2^{n-1}}\sqrt{p_j}|j\rangle \rightarrow \sum_{j=0}^{2^{n-1}}\sqrt{p_j}|j\rangle\big(\sqrt{1-\tilde{v}(x_j)}|0\rangle + \sqrt{\tilde{v}(x_j)}|1\rangle\big) =: |\chi\rangle$$

To do so, we first need to encode the result of the payoff function $\tilde{v}(x_j)$ into the target qubits. to obtain:

$$|j\rangle|0^n\rangle\rightarrow|j\rangle|\tilde{v}(x_j)\rangle$$

## Payoff function circuit

Next, we need to create our circuit to calculate the payoff function. The actual payoff function for a European option is $MAX(0, K - e^{\sigma x+(r-\sigma^2/2)t})$. 

Since building the payoff function requires creating gates for many more complex operations, we instead just use the adder that we created before. Consider this the simplest case where $S$ is the price of the security and we add the random variable encoded in $|j\rangle$ to it. Now we have:

$|j\rangle|0^n\rangle\rightarrow|j\rangle|\tilde{v}(j)\rangle$ where $\tilde{v}(j) = j + S$.

For simplicity, we'll use $S=1$. First we write a program to encode the number to be added in the qubits adjacent to the probability distribution and pad the state with an additional $l$ qubit carry register. The target register for the adder must have $l+1$ qubits to account for the possibility of overflow.

In [17]:
s = 1
R = append_number_to_state(1, n)
R += pad_with_zeros(n)
p += R
print(qvm.wavefunction(p))

(0.1178392437+0j)|00000100> + (0.6972186979+0j)|00000101> + (0.6972186979+0j)|00000110> + (0.1178392437+0j)|00000111>


We see that the number 1 is now encoded in the 3 qubit target register. Now, we perform the addition into the target register using the full quantum adder. Since we'll need to reverse these steps at the end of the program, we keep track of them in a separate program $\mathcal{R}$ so we can recover the unitary.

In [18]:
R = quantum_adder_from_state(R, n)
p = quantum_adder_from_state(p, n)

In [19]:
print(qvm.wavefunction(p))

(0.1178392437+0j)|00000100> + (0.6972186979+0j)|00001001> + (0.6972186979+0j)|00001110> + (0.1178392437+0j)|00010011>


We see that the correct result of the addition is now encoded in the 3-qubit target register, and the 2-qubit carry register has been cleared. We are now in the state $$\mathcal{G}|0^n\rangle = \sum_{j=0}^{2^{n-1}}\sqrt{p_j}|j\rangle|\tilde{v}(x_j)\rangle|0^{l}\rangle$$

and the payoff function has been successfully encoded into the target register.

## Perform controlled rotation

Now, we use the algorithm for a controlled rotation to rotate the ancilla qubit. The first step is to take $|\tilde{v}(j)\rangle \rightarrow |\sqrt{\tilde{v}(j)})\rangle$. We know that there exists a quantum circuit for implementing $\sqrt{x}$ since any classical function can be mapped into a quantum operation using reversible logic and Fredkin gates. However, implementing those gates is beyond the scope of this project. Instead, we'll implement a placeholder gate defined as $\sqrt{\tilde{v}(j)})$ to be replaced at a later date. The gate simply performs the identity on each qubit in the target register.

Now we create the circuit to perform a controlled rotation on the ancilla qubit on page 14 of "Quantum algorithm and circuit design solving the Poisson equation".

In [20]:
def nbit_sqrt(n: int):
    p = Program()
    for q in [i + n for i in range(n + 1)]:
        p += I(q)
    return p

In [21]:
A = nbit_sqrt(n)
p += A
R += A

### Controlled rotation

Now we need to create the gate for rotating the ancilla qubit with the $|\sqrt{\tilde{v}(j)}\rangle$ state as the control qubits:

$$R|\sqrt{\tilde{v}(j)})\rangle|0\rangle \rightarrow |\sqrt{\tilde{v}(j)})\rangle\big(\sqrt{1-\tilde{v}(x_j)}|0\rangle + \sqrt{\tilde{v}(x_j)}|1\rangle\big)$$

where $R$ consists of applying $l$ controlled $R_y$ rotations. 

In [22]:
def controlled_rotation(control_reg: List[int], ancilla: int) -> Program:
    p = Program()
    for i, q in enumerate(control_reg):
        theta = 1/2**i
        p += RY(theta, ancilla).controlled(q)
    return p

In [23]:
R_dagger = R.dagger()
R += controlled_rotation(control_reg, ancilla)
p += controlled_rotation(control_reg, ancilla)

In [24]:
print(qvm.wavefunction(p))

(0.1034136654+0j)|00000100> + (0.675543857+0j)|00001001> + (0.5101471604+0j)|00001110> + (0.1169198227+0j)|00010011> + (0.0564951429+0j)|00100100> + (0.1724946663+0j)|00101001> + (0.4752512887+0j)|00101110> + (0.0146915763+0j)|00110011>


We then reverse the action of the gates on control register, resulting in the desired state 

$$|\chi\rangle = \sum_{j=0}^{2^{n-1}}\sqrt{p_j}|j\rangle\big(\sqrt{1-\tilde{v}(x_j)}|0\rangle + \sqrt{\tilde{v}(x_j)}|1\rangle\big)$$

We also have the complete unitary $\mathcal{R}$, which performs the rotation of the ancilla. 

In [25]:
p += R_dagger
R += R_dagger
print(qvm.wavefunction(p))

(0.1034136654+0j)|00000000> + (0.675543857+0j)|00000001> + (0.5101471604+0j)|00000010> + (0.1169198227+0j)|00000011> + (0.0564951429+0j)|00100000> + (0.1724946663+0j)|00100001> + (0.4752512887+0j)|00100010> + (0.0146915763+0j)|00100011>


We can confirm that the unitary $\mathcal{R}$ produces the desired state by applying it to the state produced by the unitary $\mathcal{G}$ created above and confirming that it matches our final wavefunction.

In [26]:
pG = Program()
pG += G_def
pG += G(0, 1, 2, 3, 4, 5, 6, 7)
print("Original probability distribution")
print(qvm.wavefunction(pG))
pG += R
print("\n")
print("State after rotation")
print(qvm.wavefunction(pG))

Original probability distribution
(0.1178392437+0j)|00000000> + (0.6972186979+0j)|00000001> + (0.6972186979+0j)|00000010> + (0.1178392437+0j)|00000011>


State after rotation
(0.1034136654+0j)|00000000> + (0.675543857+0j)|00000001> + (0.5101471604+0j)|00000010> + (0.1169198227+0j)|00000011> + (0.0564951429+0j)|00100000> + (0.1724946663+0j)|00100001> + (0.4752512887+0j)|00100010> + (0.0146915763+0j)|00100011>


The form of the state in quantum memory is $|\chi\rangle = |j\rangle|0^{l+1}\rangle|\big(\sqrt{1-\tilde{v}(x_j)}|0\rangle + \sqrt{\tilde{v}(x_j)}|1\rangle\big)|0^l\rangle$

We can now use phase estimation to calculate the expectation value of $\mu = \langle\chi|\big(\mathcal{I_{2^{2l+1}}}\otimes|1\rangle\langle 1| \otimes \mathcal{I_{2^{l}}}\big)|\chi\rangle$ according to (21).

## Create Q for phase estimation

To estimate the expectation value, we use the unitary $\mathcal{Q}$ defined in (27).

### Define V

We define the unitary $\mathcal{V} = \mathcal{I}_{2^{3l+2}}-2\big(\mathcal{I_{2^{2l+1}}}\otimes|1\rangle\langle 1| \otimes \mathcal{I_{2^{l}}}\big)$

In [27]:
one = np.matrix([[0,1]], dtype=np.complex_)
one_h = one.getH()
projector = np.matmul(one_h, one)

V_unitary = np.identity(2**(3*n+2)) - 2*np.kron(np.kron(np.identity(2**(2*n+1)), projector), np.identity(2**n))
print(V_unitary)

[[ 1.+0.j  0.+0.j  0.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 ...
 [ 0.+0.j  0.+0.j  0.+0.j ... -1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...  0.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...  0.+0.j  0.+0.j -1.+0.j]]


### Define U

To define $\mathcal{U} = \mathcal{F}\mathcal{Z}\mathcal{F}^\dagger$, we first define $\mathcal{Z} = \mathcal{I}_{2^{3l+2}} - 2|0^{3n+2}\rangle\langle 0^{3n+2}|$

In [28]:
projector = np.zeros((2**(3*n+2),2**(3*n+2)), dtype=np.complex_)
projector[0][0] = 1

Z_unitary = np.identity(2**(3*n+2)) - 2*projector
print(Z_unitary)
print(len(Z_unitary))

[[-1.+0.j  0.+0.j  0.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 ...
 [ 0.+0.j  0.+0.j  0.+0.j ...  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...  0.+0.j  0.+0.j  1.+0.j]]
256


We define $\mathcal{F}$ by extracting the program unitary from $\mathcal{R}$ and composing with $\mathcal{G}$ according to (20). 

In [29]:
R_unitary = np.matrix(program_unitary(R, 3*n + 2), dtype=np.complex_)
F_unitary = np.matmul(R_unitary, G_unitary)
F_unitary_dagger = F_unitary.getH()

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

(shapes (1,), (2,) mismatch)
 x: array([5])
 y: array([2, 5])

Unfortunately, this is where the fun ends. We were not able to identify the bug that prevents us from retrieving the program unitary from $\mathcal{R}$. Nevertheless, we proceed with the rest of the algorithm as if we were able to do so. 

If we had been able to construct $\mathcal{R}$, we could finally create $\mathcal{U}$ through matrix multiplication.

In [92]:
U_unitary = np.matmul(np.matmul(F_unitary, Z_unitary), F_unitary_dagger)

### Define Q

We use the transformation $\mathcal{Q} = \mathcal{U}\mathcal{V}\mathcal{U}\mathcal{V}$ for phase estimation.

In [93]:
Q_unitary = np.matmul(np.matmul(U_unitary, V_unitary), np.matmul(U_unitary, V_unitary))

## Phase estimation

For phase estimation on $\mathcal{Q}$, we use Grove's phase estimation module.

In [98]:
precision = 8
phase = phase_estimation(Q_unitary, precision)
output = qvm.run(phase)
wavefunction = qvm.wavefunction(phase)

In [99]:
def binary_frac_to_dec(b: List[int]):
    dec = 0
    for i, bit in enumerate(reversed(b)):
        dec += bit/2**(i+1)
    return dec

In [100]:
phase_factor = binary_frac_to_dec(output[0])
print(phase_factor)

0.0


In this case, phase estimation returned a phase of 0, although this is not a meaningful result since the rotation gate was not implemented. Once the rotation is implemented correctly, this step should return the correct phase angle.

### Expectation value

According to (24), $1-2\mu = \cos(\theta/2)$, so using phase factor, we can estimate the expectation value.

In [104]:
theta = 2*np.pi*phase_factor
mu = (1 - np.cos(theta/2))/2
print(mu)

0.0


Of course, this result is meaningless given that the payout function and rotation gates were not implemented. But those are coming!

# Appendix

Original carry and sum functions accept a list of QubitPlaceholders instead of indices.

In [7]:
def quantum_carry(q: List[QubitPlaceholder]) -> Program:
    p = Program()
    p += CCNOT(q[1], q[2], q[3])
    p += CNOT(q[1], q[2])
    p += CCNOT(q[0], q[2], q[3])
    return p

def reverse_carry(q: List[QubitPlaceholder]) -> Program:
    p = Program()
    p += CCNOT(q[0], q[2], q[3])
    p += CNOT(q[1], q[2])
    p += CCNOT(q[1], q[2], q[3])
    return p

def quantum_sum(q: List[QubitPlaceholder]) -> Program:
    p = Program()
    p += CNOT(q[1], q[2])
    p += CNOT(q[0], q[2])
    return p

An alternate quantum adder that takes two integers as input and returns a quantum program to perform addition.

In [102]:
#implement quantum adder

def quantum_adder_int(a: int, b: int, n: int):
    assert a < 2**n, "a uses too many bits"
    assert b < 2**n, "b uses too many bits"
    
    state_a = QubitPlaceholder.register(n)
    state_b = QubitPlaceholder.register(n + 1)
    state_c = QubitPlaceholder.register(n)
    
    #print("Length of state b is {}".format(len(state_b)))
    #print("Length of state b is {}".format(len(state_b)))
    
    p = int_to_state(a, state_a)
    p += int_to_state(b, state_b)
    p += int_to_state(0, state_c) #assign all c qubits to 0 to preserve indexing 
    
    for i in range(n):
        if i == n-1: 
            p += quantum_carry([state_c[i], state_a[i], state_b[i], state_b[i+1]])
        else:
            p += quantum_carry([state_c[i], state_a[i], state_b[i], state_c[i+1]])
            
    #print("First round of carries finished\n")
    
    p += CNOT(state_a[n-1], state_b[n-1])
    p += quantum_sum([state_c[n-1], state_a[n-1], state_b[n-1]])
    
    #print("Begin reversal\n")
    
    for i in reversed(range(n-1)):
        p += reverse_carry([state_c[i], state_a[i], state_b[i], state_c[i+1]])
        p += quantum_sum([state_c[i], state_a[i], state_b[i]])
    
    return address_qubits(p)


Helper functions for checking accuracy of the quantum adder. 

In [103]:
def check_adder(a: int, b: int, n: int): 
    result = a+b
    p = Program()
    p += quantum_adder_int(a, b, n)
    print(qvm.wavefunction(p))
    ro = p.declare('ro', 'BIT', 2*n+1)
    for i in range(2*n+1):
        p += MEASURE(i+n, ro[i])
    binary = qvm.run(p)
    binary = [str(i) for i in reversed(binary[0])]
    binary = ''.join(binary)
    decimal_result = int(binary, 2)
    return decimal_result
        
def check_all_combinations_int(n: int):
    for i in range(2**n):
        for j in range(2**n):
            result = check_adder(i, j, n)
            output = i + j
            if result == output:
                print("{} + {} = {}. Success!".format(i, j, result))
            else: 
                print("{} + {} != {}. Failure!".format(i, j, result))
            